In [1]:
import sys
sys.path.append("../../")

In [ ]:
from dvalib import indexer

In [ ]:
facenet = indexer.FacenetIndexer()

In [ ]:
temp2 = facenet.nearest('/Users/aub3/media/queries/1.png')

In [ ]:
vec = facenet.apply('/Users/aub3/media/queries/1.png')

In [ ]:
vec[0,:5]

In [ ]:
facenet.load_index('/Users/aub3/media/')

In [ ]:
facenet.index.shape

In [ ]:
from scipy import spatial

In [ ]:
facenet.index[:1,:].shape

In [ ]:
facenet.index.shape

In [ ]:
distance = spatial.distance.cdist(vec,facenet.index)

In [ ]:
distance

In [ ]:
distance.argsort()

In [ ]:
facenet.files[6]

In [ ]:
for k in range(35):
    print sum(facenet.index[k,:128]-vec[0,:128])

### Explore dvalib which implements operations to perform indexing and detection

In [2]:
from dvalib import indexer
temp = indexer.INDEXERS['inception'].apply('person.jpg')
# temp2 = indexer.INDEXERS['alex'].apply('person.jpg')

In [3]:
print temp.shape

(1, 2048)


## Following steps requires dva django settings.py to be configured with DB, RabbitMQ

In [ ]:
import django,os,glob
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

## Extract frames

In [ ]:
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.views import handle_uploaded_file, handle_youtube_video
from dvaapp.models import Video
from dvaapp.tasks import extract_frames, perform_indexing, perform_detection
for fname in glob.glob('../../tests/ci/*.mp4'):
    name = fname.split('/')[-1].split('.')[0]
    f = SimpleUploadedFile(fname, file(fname).read(), content_type="video/mp4")
    handle_uploaded_file(f, name, False)
for fname in glob.glob('../../tests/*.zip'):
    name = fname.split('/')[-1].split('.')[0]
    f = SimpleUploadedFile(fname, file(fname).read(), content_type="application/zip")
    handle_uploaded_file(f, name)
handle_youtube_video('tomorrow never dies', 'https://www.youtube.com/watch?v=gYtz5sw98Bc')
for v in Video.objects.all():
    extract_frames(v.pk)
    perform_indexing(v.pk)
    perform_detection(v.pk)

##  Query data stored in database via Django ORM

In [ ]:
from dvaapp.models import Detection,Video
for v in Video.objects.all():
    print v.name
    break
for d in Detection.objects.all():
    print d.object_name,d.x,d.y,d.h,d.w,d.confidence
    break

In [ ]:
from dvalib import pyscenecustom
save_image_prefix = "../../tests/"
manager = pyscenecustom.manager.SceneManager(save_image_prefix=save_image_prefix)
path = "../../tests/Goldeneye.mp4"
pyscenecustom.detect_scenes_file(path,manager)